## Recommendation System using Collaborative Filtering Based

In [2]:
import pandas as pd
import numpy as np

In [3]:
bg_df = pd.read_csv("../../Datasource/games_detailed_info.csv")
bg_df

C:\Users\COMPUTER\AppData\Local\Temp\ipykernel_6740\1990694590.py:1: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  bg_df = pd.read_csv("../../Datasource/games_detailed_info.csv")


,Unnamed: 0,type,id,thumbnail,image,primary,alternate,description,yearpublished,minplayers,...,War Game Rank,Customizable Rank,Children's Game Rank,RPG Item Rank,Accessory Rank,Video Game Rank,Amiga Rank,Commodore 64 Rank,Arcade Rank,Atari ST Rank
0,0,boardgame,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,Pandemic,"['EPIZOotic', 'Pandemia', 'Pandemia 10 Anivers...","In Pandemic, several virulent diseases have br...",2008,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,boardgame,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,Carcassonne,"['Carcassonne Jubilee Edition', 'Carcassonne: ...",Carcassonne is a tile-placement game in which ...,2000,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,boardgame,13,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...,https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...,Catan,"['CATAN', 'Catan (Колонизаторы)', 'Catan telep...","In CATAN (formerly The Settlers of Catan), pla...",1995,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,boardgame,68448,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...,7 Wonders,"['7 csoda', '7 Cudów Świata', '7 divů světa', ...",You are the leader of one of the 7 great citie...,2010,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,boardgame,36218,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...,https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...,Dominion,"['Dominion: Basisspiel', 'Dominion: In naam va...","&quot;You are a monarch, like your parents bef...",2008,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21626,21626,boardgame,296892,https://cf.geekdo-images.com/d37HVSmlYM2ZQpaKD...,https://cf.geekdo-images.com/d37HVSmlYM2ZQpaKD...,Sacred Rites,NaN,It has come time to honor the traditions of th...,2020,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21627,21627,boardgame,217378,https://cf.geekdo-images.com/BDS9Ndm2gb8QyE-i1...,https://cf.geekdo-images.com/BDS9Ndm2gb8QyE-i1...,BABEL,NaN,BABEL is a balancing game in which players mak...,2016,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21628,21628,boardgame,18063,https://cf.geekdo-images.com/NRIjTeX6AbdhOp-w1...,https://cf.geekdo-images.com/NRIjTeX6AbdhOp-w1...,Goldrush,"['Arizona', 'Guldfeber', 'Kultakuume']",From the rules:&#10;&#10;Players are prospecto...,1970,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21629,21629,boardgame,10052,https://cf.geekdo-images.com/EGOJ027F8r3Nln1r0...,https://cf.geekdo-images.com/EGOJ027F8r3Nln1r0...,Bongo Kongo,NaN,Each player takes a position adjacent to one o...,1989,2,...,NaN,NaN,350,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
bg_df = bg_df[["id", "primary"]]
bg_df  = bg_df.rename(columns={"id" : "boardgame_id", "primary" : "boardgame_name"})
bg_df

,boardgame_id,boardgame_name
0,30549,Pandemic
1,822,Carcassonne
2,13,Catan
3,68448,7 Wonders
4,36218,Dominion
...,...,...
21626,296892,Sacred Rites
21627,217378,BABEL
21628,18063,Goldrush
21629,10052,Bongo Kongo


In [5]:
ratings_df = pd.read_csv("../Datasets/ratings.csv")
ratings_df

,user_name,rating,comment,boardgame_id,boardgame_name
0,Staxston,9.2,Only game I really play solo.,171835,Posthuman
1,Sheylon,3.0,Cute pieces but this isn't much of a game. Rol...,5824,The Kids of Catan
2,jens_hoppe,3.0,Initial impression. Takes heavy-weight German-...,25554,Notre Dame
3,bcnevan,5.0,Shores of Tripoli succeeds in some ways. It is...,237860,The Shores of Tripoli
4,ruelgaviola,8.0,Donated,17329,Animal Upon Animal
...,...,...,...,...,...
673718,Andy Parsons,5.0,I found the theme of Say Bye... quite appealin...,140795,Say Bye to the Villains
673719,PHIL4477,4.0,Another trivia game. I bought it because the b...,32441,Bezzerwizzer
673720,EmeraldYam,7.0,(As of [b]3[/b] plays) Quite original card-bas...,125315,FlowerFall
673721,PaleHorseRider,6.0,Good version of Apples to Apples. It has cons...,104234,Sour Apples to Apples


In [6]:
ratings_df["user_name"].value_counts() 

user_name
Nap16          776
Walt Mulder    661
Pandorzecza    581
PBrennan       569
punkin312      541
              ... 
maisherly        1
J69AGENT         1
aschrad39        1
dlydle           1
vaguereiz        1
Name: count, Length: 89840, dtype: int64

In [7]:
ratings_df["user_name"].unique().shape

(89840,)

In [8]:
user_count_rating = ratings_df["user_name"].value_counts()  > 50
user_count_rating.value_counts()

count
False    87752
True      2088
Name: count, dtype: int64

In [9]:
# User - User interaction
user_count_rating[user_count_rating].shape

(2088,)

In [10]:
index_user = user_count_rating[user_count_rating].index
index_user

Index(['Nap16', 'Walt Mulder', 'Pandorzecza', 'PBrennan', 'punkin312',
       'larryjrice', 'rseater', 'visard', 'snoozefest', 'Pozman',
       ...
       'revengeisnotjustice', 'thnikkaman', 'denis_1929', 'cktjharris',
       'propane train', 'ubergigglefritz', 'pronkjewail', 'rumble', 'zackt',
       'Canales'],
      dtype='object', name='user_name', length=2088)

In [11]:
ratings = ratings_df[ratings_df["user_name"].isin(index_user)]
ratings

,user_name,rating,comment,boardgame_id,boardgame_name
1,Sheylon,3.0,Cute pieces but this isn't much of a game. Rol...,5824,The Kids of Catan
2,jens_hoppe,3.0,Initial impression. Takes heavy-weight German-...,25554,Notre Dame
3,bcnevan,5.0,Shores of Tripoli succeeds in some ways. It is...,237860,The Shores of Tripoli
5,brianeikunst,6.0,fairly abstract & mostly tactical. need at le...,30241,Patrician
6,Stas,7.5,Last played: Mar 2016,35497,Fauna
...,...,...,...,...,...
673706,Thrylos,8.0,This is an underrated game. It is actually a v...,26047,Sid Meier's Civilization: The Card Game
673716,wkover,6.0,"I'm generally not keen on party games, but thi...",20100,Wits & Wagers
673718,Andy Parsons,5.0,I found the theme of Say Bye... quite appealin...,140795,Say Bye to the Villains
673719,PHIL4477,4.0,Another trivia game. I bought it because the b...,32441,Bezzerwizzer


In [12]:
ratings_bg = ratings[["user_name", "rating", "boardgame_id", "boardgame_name"]]
ratings_bg

,user_name,rating,boardgame_id,boardgame_name
1,Sheylon,3.0,5824,The Kids of Catan
2,jens_hoppe,3.0,25554,Notre Dame
3,bcnevan,5.0,237860,The Shores of Tripoli
5,brianeikunst,6.0,30241,Patrician
6,Stas,7.5,35497,Fauna
...,...,...,...,...
673706,Thrylos,8.0,26047,Sid Meier's Civilization: The Card Game
673716,wkover,6.0,20100,Wits & Wagers
673718,Andy Parsons,5.0,140795,Say Bye to the Villains
673719,PHIL4477,4.0,32441,Bezzerwizzer


In [13]:
# Tính số lượng ratings từ các user
number_of_ratings_bg = ratings_bg.groupby("boardgame_name")["rating"].count().reset_index()
number_of_ratings_bg

,boardgame_name,rating
0,"""La Garde recule!""",2
1,"""Scratch One Flat Top!""",1
2,"""Tarleton's Quarter!""",3
3,#MyLife,2
4,"'CA' Tactical Naval Warfare in the Pacific, 19...",2
...,...,...
16686,ドキッと！アイス (Dokitto! Ice),1
16687,ルイス (luz),1
16688,白と黒でトリテ (Trick-Taking in Black and White),1
16689,目撃者たちの夜 (Witness Night),1


In [14]:
# Tính trung bình ratings từ các user
mean_of_ratings_bg = ratings_bg.groupby("boardgame_name")["rating"].mean().reset_index()
mean_of_ratings_bg

,boardgame_name,rating
0,"""La Garde recule!""",4.000000
1,"""Scratch One Flat Top!""",5.000000
2,"""Tarleton's Quarter!""",6.000000
3,#MyLife,4.500000
4,"'CA' Tactical Naval Warfare in the Pacific, 19...",3.500000
...,...,...
16686,ドキッと！アイス (Dokitto! Ice),7.300000
16687,ルイス (luz),6.000000
16688,白と黒でトリテ (Trick-Taking in Black and White),7.000000
16689,目撃者たちの夜 (Witness Night),6.000000


In [15]:
mean_and_number_ratings = number_of_ratings_bg.merge(mean_of_ratings_bg, on= "boardgame_name")
mean_and_number_ratings

,boardgame_name,rating_x,rating_y
0,"""La Garde recule!""",2,4.000000
1,"""Scratch One Flat Top!""",1,5.000000
2,"""Tarleton's Quarter!""",3,6.000000
3,#MyLife,2,4.500000
4,"'CA' Tactical Naval Warfare in the Pacific, 19...",2,3.500000
...,...,...,...
16686,ドキッと！アイス (Dokitto! Ice),1,7.300000
16687,ルイス (luz),1,6.000000
16688,白と黒でトリテ (Trick-Taking in Black and White),1,7.000000
16689,目撃者たちの夜 (Witness Night),1,6.000000


In [16]:
mean_and_number_ratings = mean_and_number_ratings.rename(columns={"rating_x" : "rating_count", "rating_y" : "rating_mean"})
mean_and_number_ratings

,boardgame_name,rating_count,rating_mean
0,"""La Garde recule!""",2,4.000000
1,"""Scratch One Flat Top!""",1,5.000000
2,"""Tarleton's Quarter!""",3,6.000000
3,#MyLife,2,4.500000
4,"'CA' Tactical Naval Warfare in the Pacific, 19...",2,3.500000
...,...,...,...
16686,ドキッと！アイス (Dokitto! Ice),1,7.300000
16687,ルイス (luz),1,6.000000
16688,白と黒でトリテ (Trick-Taking in Black and White),1,7.000000
16689,目撃者たちの夜 (Witness Night),1,6.000000


In [17]:
### Kết hợp count_rating with the original dataset
ratings_bg_cf = ratings_bg.merge(mean_and_number_ratings, on = "boardgame_name")
ratings_bg_cf

,user_name,rating,boardgame_id,boardgame_name,rating_count,rating_mean
0,Sheylon,3.0,5824,The Kids of Catan,20,4.41
1,akapastorguy,6.0,5824,The Kids of Catan,20,4.41
2,paulkessels,5.0,5824,The Kids of Catan,20,4.41
3,boenke,5.0,5824,The Kids of Catan,20,4.41
4,joebelanger,1.0,5824,The Kids of Catan,20,4.41
...,...,...,...,...,...,...
209427,davestephens,7.0,140829,Hunter's Guild: The Vampire Forest,1,7.00
209428,LordBalder,7.0,225235,First Snow,1,7.00
209429,CharlieWonka,6.0,18562,World Championship Dodge Ball,1,6.00
209430,Wolverutto,8.0,194194,Pędzące Ślimaki,1,8.00


In [18]:
ratings_bg_cf["rating_count"].unique().max()

359

In [19]:
ratings_bg_cf = ratings_bg_cf[ratings_bg_cf["rating_count"] > 30]
ratings_bg_cf

,user_name,rating,boardgame_id,boardgame_name,rating_count,rating_mean
20,jens_hoppe,3.0,25554,Notre Dame,192,7.111125
21,KrisVerbeeck,7.0,25554,Notre Dame,192,7.111125
22,bazik123,6.0,25554,Notre Dame,192,7.111125
23,Chris Coyote,6.5,25554,Notre Dame,192,7.111125
24,Orph,7.0,25554,Notre Dame,192,7.111125
...,...,...,...,...,...,...
193627,Michiel,4.5,95,Igel Ärgern,31,6.041935
193628,dturnerfish,6.5,95,Igel Ärgern,31,6.041935
193629,jayjonbeach,3.0,95,Igel Ärgern,31,6.041935
193630,Defdamesdompi,6.5,95,Igel Ärgern,31,6.041935


In [20]:
import os
path = "../Datasets/cfb_dataset.csv"
if os.path.exists(path):
    name_file = path.split('/')[-1]
    print(f"File {name_file} đã tồn tại. Bắt đầu xóa file cũ....")
    os.remove(path)
    
ratings_bg_cf.to_csv(path, index=False)
folder_file = path.split('/')[-2]
print(f"File {name_file} đã được thêm vào thư mục {folder_file}.")


File cfb_dataset.csv đã tồn tại. Bắt đầu xóa file cũ....
File cfb_dataset.csv đã được thêm vào thư mục Datasets.
